### 1. What is the purpose of this experiment?
LGBMのkernelにおける学習時間を計測する
### 2. Why do you this?
CPUで攻めるかGPUで攻めるか判断する.GPUの場合もLGBMを使うか判断する
### 3. Where are the points of technology and techniques?
LGBM
### 4. How do you validate the effectiveness?
処理時間
### 5. What will you do next?
NN * FE

In [2]:
is_local = False
seed = 1208

%load_ext autoreload
%autoreload 2
from tqdm import tqdm
import numpy as np
import pandas as pd
import time
from contextlib import contextmanager
import datetime
import sys
import re
import gc
import glob
import pickle as pkl
import os

from joblib import Parallel, delayed
from multiprocessing import Pool
import multiprocessing
def pararell_process(func, arg_list, cpu_cnt=multiprocessing.cpu_count()):
    process = Pool(cpu_cnt)
    callback = process.map_async(func, arg_list).get(600)
    process.close()
    process.terminate()
    return callback

def mkdir_func(path):
    try:
        os.stat(path)
    except:
        os.mkdir(path)

from logging import StreamHandler, DEBUG, Formatter, FileHandler, getLogger
def logger_func():
    logger = getLogger(__name__)
    log_fmt = Formatter('%(asctime)s %(name)s %(lineno)d [%(levelname)s]\
    [%(funcName)s] %(message)s ')
    handler = StreamHandler()
    handler.setLevel('INFO')
    handler.setFormatter(log_fmt)
    logger.addHandler(handler)

    mkdir_func('../output')
    handler = FileHandler('../output/py_train.py.log', 'a')
    handler.setLevel(DEBUG)
    handler.setFormatter(log_fmt)
    logger.setLevel(DEBUG)
    logger.addHandler(handler)

    logger.info('start')

    return logger
logger = logger_func()

st_time = time.time()
@contextmanager
def timer(name):
    """
    Taken from Konstantin Lopuhin https://www.kaggle.com/lopuhin
    in script named : Mercari Golf: 0.3875 CV in 75 LOC, 1900 s
    https://www.kaggle.com/lopuhin/mercari-golf-0-3875-cv-in-75-loc-1900-s
    """
    t0 = time.time()
    yield
    print(f'[{name}] done in {time.time() - t0:.0f} s')
    
# NLP
from wordcloud import WordCloud, STOPWORDS
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string

key = 'qid'
qt = 'question_text'
target = 'target'

2018-11-26 08:56:37,404 __main__ 51 [INFO]    [logger_func] start 


### Cleansing Method

In [4]:
# Contraction replacement patterns
cont_patterns = [
    (b'(W|w)on\'t', b'will not'),
    (b'(C|c)an\'t', b'can not'),
    (b'(I|i)\'m', b'i am'),
    (b'(A|a)in\'t', b'is not'),
    (b'(\w+)\'ll', b'\g<1> will'),
    (b'(\w+)n\'t', b'\g<1> not'),
    (b'(\w+)\'ve', b'\g<1> have'),
    (b'(\w+)\'s', b'\g<1> is'),
    (b'(\w+)\'re', b'\g<1> are'),
    (b'(\w+)\'d', b'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

def prepare_for_char_n_gram(text):
    """ Simple text clean up process"""
    # 1. Go to lower case (only good for english)
    # Go to bytes_strings as I had issues removing all \n in r""
    clean = bytes(text.lower(), encoding="utf-8")
    # 2. Drop \n and  \t
    clean = clean.replace(b"\n", b" ")
    clean = clean.replace(b"\t", b" ")
    clean = clean.replace(b"\b", b" ")
    clean = clean.replace(b"\r", b" ")
    # 3. Replace english contractions
    for (pattern, repl) in patterns:
        clean = re.sub(pattern, repl, clean)
    # 4. Drop puntuation
    # I could have used regex package with regex.sub(b"\p{P}", " ")
    exclude = re.compile(b'[%s]' % re.escape(bytes(string.punctuation, encoding='utf-8')))
    clean = b" ".join([exclude.sub(b'', token) for token in clean.split()])
    # 5. Drop numbers - as a scientist I don't think numbers are toxic ;-)
    clean = re.sub(b"\d+", b" ", clean)
    # 6. Remove extra spaces - At the end of previous operations we multiplied space accurences
    clean = re.sub(b'\s+', b' ', clean)
    # Remove ending space if any
    clean = re.sub(b'\s+$', b'', clean)
    # 7. Now replace words by words surrounded by # signs
    # e.g. my name is bond would become #my# #name# #is# #bond#
    # clean = re.sub(b"([a-z]+)", b"#\g<1>#", clean)
#     clean = re.sub(b" ", b"# #", clean)  # Replace space
#     clean = b"#" + clean + b"#"  # add leading and trailing #

    return str(clean, 'utf-8')

def count_regexp_occ(regexp="", text=None):
    """ Simple way to get the number of occurence of a regex"""
    return len(re.findall(regexp, text))

def cleansing_text(text, remove_stopwords=True):

    # Convert words to lower case and split them
    text = re.sub("_", " ", text, flags=re.IGNORECASE)
    text = text.lower().split()
    regex_num = re.compile(u"[0-9０-９]")

    # Optionally, remove stop words
    if remove_stopwords:
        stops = STOPWORDS
        text = [w for w in text if (not w in stops) and not(regex_num.match(w))]

    text = " ".join(text)

    # Clean the text
    text = re.sub(" whats ", " what is ", text, flags=re.IGNORECASE)
    text = re.sub("[c-fC-F]\:\/", " disk ", text)
    text = re.sub("\'d", " would ", text)
    text = re.sub("\'ll", " will ", text)
    text = re.sub("\'re", " are ", text)
    text = re.sub("\'s", " ", text) # we have cases like "Sam is" or "Sam's" (i.e. his) these two cases aren't separable, I choose to compromise are kill "'s" directly
    text = re.sub("\'ve", " have ", text)
    text = re.sub("\(s\)", " ", text, flags=re.IGNORECASE)
    text = re.sub("`", "'", text) # special single quote
    text = re.sub("e-mail", " email ", text, flags=re.IGNORECASE)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub("b\.g\.", " bg ", text, flags=re.IGNORECASE)
    text = re.sub("e\.g\.", " eg ", text, flags=re.IGNORECASE)
    text = re.sub("i'm", "i am", text, flags=re.IGNORECASE)
    text = re.sub("é", "e", text)
    text = re.sub("’", "'", text) # special single quote
    text = re.sub("“", '"', text) # special double quote
    text = re.sub("…", " ", text)
    text = re.sub("？", "?", text)
    text = re.sub('(?<=[0-9])\,(?=[0-9])', "", text)
    text = re.sub('\$', " dollar ", text)
    text = re.sub('\%', " percent ", text)
    text = re.sub('\&', " and ", text)
    
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\W|^)([0-9]+)[kK](\W|$)", r"\1\g<2>000\3", text) # better regex provided by @armamut
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"what's", "what is ", text)

    # Original
    text = re.sub(r" u s ", " America ", text)
    text = re.sub("(the[\s]+|The[\s]+)?U\.S\.A\.", " America ", text, flags=re.IGNORECASE)
    text = re.sub("(the[\s]+|The[\s]+)?United State(s)?", " America ", text, flags=re.IGNORECASE)
    text = re.sub(r" (the[\s]+|The[\s]+)?US(A)? ", " America ", text)
    text = re.sub(r" UK ", " England ", text, flags=re.IGNORECASE)
    text = re.sub(r" india ", " India ", text)
    text = re.sub(r" switzerland ", " Switzerland ", text)
    text = re.sub(r" china ", " China ", text)
    text = re.sub(r" chinese ", " Chinese ", text)
    text = re.sub(r" quora ", " Quora ", text, flags=re.IGNORECASE)
    text = re.sub(r" qoura ", " Quora ", text)
    text = re.sub(r" upvote", " up vote", text, flags=re.IGNORECASE)
    text = re.sub(r" iPhone ", " phone ", text, flags=re.IGNORECASE)
    text = re.sub(r" J K ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r" J\.K\. ", " JK ", text, flags=re.IGNORECASE)
    text = re.sub(r"(?=[a-zA-Z])ig ", "ing ", text)
    
    return text

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have" }
mispell_dict = {'colour': 'color', 'centre': 'center', 'favourite': 'favorite', 'travelling': 'traveling', 'counselling': 'counseling', 'theatre': 'theater', 'cancelled': 'canceled', 'labour': 'labor', 'organisation': 'organization', 'wwii': 'world war 2', 'citicise': 'criticize', 'youtu ': 'youtube ', 'Qoura': 'Quora', 'sallary': 'salary', 'Whta': 'What', 'narcisist': 'narcissist', 'howdo': 'how do', 'whatare': 'what are', 'howcan': 'how can', 'howmuch': 'how much', 'howmany': 'how many', 'whydo': 'why do', 'doI': 'do I', 'theBest': 'the best', 'howdoes': 'how does', 'mastrubation': 'masturbation', 'mastrubate': 'masturbate', "mastrubating": 'masturbating', 'pennis': 'penis', 'Etherium': 'Ethereum', 'narcissit': 'narcissist', 'bigdata': 'big data', '2k17': '2017', '2k18': '2018', 'qouta': 'quota', 'exboyfriend': 'ex boyfriend', 'airhostess': 'air hostess', "whst": 'what', 'watsapp': 'whatsapp', 'demonitisation': 'demonetization', 'demonitization': 'demonetization', 'demonetisation': 'demonetization'}
punct_mapping = {"‘": "'", "₹": "e", "´": "'", "°": "", "€": "e", "™": "tm", "√": " sqrt ", "×": "x", "²": "2", "—": "-", "–": "-", "’": "'", "_": "-", "`": "'", '“': '"', '”': '"', '“': '"', "£": "e", '∞': 'infinity', 'θ': 'theta', '÷': '/', 'α': 'alpha', '•': '.', 'à': 'a', '−': '-', 'β': 'beta', '∅': '', '³': '3', 'π': 'pi', }    
def clean_map_text(x):
    for dic in [contraction_mapping, mispell_dict, punct_mapping]:
        for word in dic.keys():
            x = x.replace(word, dic[word])
    return x

def _get_mispell(mispell_dict):
    mispell_re = re.compile('(%s)' % '|'.join(mispell_dict.keys()))
    return mispell_dict, mispell_re

mispell_dict = {'colour':'color',
                'centre':'center',
                'didnt':'did not',
                'doesnt':'does not',
                'isnt':'is not',
                'shouldnt':'should not',
                'favourite':'favorite',
                'travelling':'traveling',
                'counselling':'counseling',
                'theatre':'theater',
                'cancelled':'canceled',
                'labour':'labor',
                'organisation':'organization',
                'wwii':'world war 2',
                'citicise':'criticize',
                'instagram': 'social medium',
                'whatsapp': 'social medium',
                'snapchat': 'social medium'
                }
mispellings, mispellings_re = _get_mispell(mispell_dict)

def replace_typical_misspell(text):
    def replace(match):
        return mispellings[match.group(0)]
    return mispellings_re.sub(replace, text)

### Data Load

In [6]:
def quara_load_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')
    return train, test

train, test = quara_load_data()
# Load id Text List
train_id_list = list(train[key].values)
test_id_list = list(test[key].values)
train_text_list = list(train[qt].values)
test_text_list = list(test[qt].values)
id_list = train_id_list + test_id_list
text_list = train_text_list + test_text_list

raw_trn_idx = list(train.index)
raw_test_idx = list(test.index)
y = train[target]

del train, test
gc.collect()

print(train_text_list[:10])

['How did Quebec nationalists see their province as a nation in the 1960s?', 'Do you have an adopted dog, how would you encourage people to adopt and not shop?', 'Why does velocity affect time? Does velocity affect space geometry?', 'How did Otto von Guericke used the Magdeburg hemispheres?', 'Can I convert montra helicon D to a mountain bike by just changing the tyres?', 'Is Gaza slowly becoming Auschwitz, Dachau or Treblinka for Palestinians?', 'Why does Quora automatically ban conservative opinions when reported, but does not do the same for liberal views?', 'Is it crazy if I wash or wipe my groceries off? Germs are everywhere.', 'Is there such a thing as dressing moderately, and if so, how is that different than dressing modestly?', 'Is it just me or have you ever been in this phase wherein you became ignorant to the people you once loved, completely disregarding their feelings/lives so you get to have something go your way and feel temporarily at ease. How did things change?']


### Feature Engineering 1

In [ ]:
with timer("Basic Feature Engineering"):
    train["num_words"] = train["question_text"].apply(lambda x: len(str(x).split()))
    test["num_words"] = test["question_text"].apply(lambda x: len(str(x).split()))
    
    ## Number of unique words in the text ## 
    train["num_unique_words"] = train["question_text"].apply(lambda x: len(set(str(x).split())))
    test["num_unique_words"] = test["question_text"].apply(lambda x: len(set(str(x).split())))
    
    ## Number of characters in the text ##
    train["num_chars"] = train["question_text"].apply(lambda x: len(str(x)))
    test["num_chars"] = test["question_text"].apply(lambda x: len(str(x)))
    
    ## Number of stopwords in the text ##
    train["num_stopwords"] = train["question_text"].apply(lambda x: len([w for w in str(x).split() if w in STOPWORDS ]))
    test["num_stopwords"] = test["question_text"].apply(lambda x: len([w for w in str(x).split() if w in STOPWORDS ]))
    
    ## Number of punctuations in the text ##
    train["num_punctuations"] = train["question_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    test["num_punctuations"] = test["question_text"].apply(lambda x: len([c for c in str(x) if c in string.punctuation]))
    
    ## Number of title case words in the text ##
    train["num_words_upper"] = train["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
    test["num_words_upper"] = test["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
    
    ## Number of title case words in the text ##
    train["num_words_upper"] = train["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
    test["num_words_upper"] = test["question_text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
    
    ## Number of title case words in the text ##
    train["num_words_lower"] = train["question_text"].apply(lambda x: len([w for w in str(x).split() if w.islower()]))
    test["num_words_lower"] = test["question_text"].apply(lambda x: len([w for w in str(x).split() if w.islower()]))
    
    ## Number of upper chars in the text ##
    train["num_chars_upper"] = train["question_text"].apply(lambda x: len([w for w in str(x) if w.isupper()]))
    test["num_chars_upper"] = test["question_text"].apply(lambda x: len([w for w in str(x) if w.isupper()]))
    
    ## Number of lower chars in the text ##
    train["num_chars_lower"] = train["question_text"].apply(lambda x: len([w for w in str(x) if w.islower()]))
    test["num_chars_lower"] = test["question_text"].apply(lambda x: len([w for w in str(x) if w.islower()]))
    
    ## Number of title case words in the text ##
    train["num_words_title"] = train["question_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
    test["num_words_title"] = test["question_text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
    
    ## Average length of the words in the text ##
    train["mean_word_len"] = train["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
    test["mean_word_len"] = test["question_text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
    
    ## Max length of the words in the text ##
    train["max_word_len"] = train["question_text"].apply(lambda x: np.max([len(w) for w in str(x).split()]))
    test["max_word_len"] = test["question_text"].apply(lambda x: np.max([len(w) for w in str(x).split()]))
    
    ## Min length of the words in the text ##
    train["min_word_len"] = train["question_text"].apply(lambda x: np.min([len(w) for w in str(x).split()]))
    test["min_word_len"] = test["question_text"].apply(lambda x: np.min([len(w) for w in str(x).split()]))

### Feature Engineering 2

In [ ]:
# The better written the code, the easier the copy pasta

# Contraction replacement patterns
cont_patterns = [
    (b'(W|w)on\'t', b'will not'),
    (b'(C|c)an\'t', b'can not'),
    (b'(I|i)\'m', b'i am'),
    (b'(A|a)in\'t', b'is not'),
    (b'(\w+)\'ll', b'\g<1> will'),
    (b'(\w+)n\'t', b'\g<1> not'),
    (b'(\w+)\'ve', b'\g<1> have'),
    (b'(\w+)\'s', b'\g<1> is'),
    (b'(\w+)\'re', b'\g<1> are'),
    (b'(\w+)\'d', b'\g<1> would'),
]
patterns = [(re.compile(regex), repl) for (regex, repl) in cont_patterns]

def prepare_for_char_n_gram(text):
    """ Simple text clean up process"""
    # 1. Go to lower case (only good for english)
    # Go to bytes_strings as I had issues removing all \n in r""
    clean = bytes(text.lower(), encoding="utf-8")
    # 2. Drop \n and  \t
    clean = clean.replace(b"\n", b" ")
    clean = clean.replace(b"\t", b" ")
    clean = clean.replace(b"\b", b" ")
    clean = clean.replace(b"\r", b" ")
    # 3. Replace english contractions
    for (pattern, repl) in patterns:
        clean = re.sub(pattern, repl, clean)
    # 4. Drop puntuation
    # I could have used regex package with regex.sub(b"\p{P}", " ")
    exclude = re.compile(b'[%s]' % re.escape(bytes(string.punctuation, encoding='utf-8')))
    clean = b" ".join([exclude.sub(b'', token) for token in clean.split()])
    # 5. Drop numbers - as a scientist I don't think numbers are toxic ;-)
    clean = re.sub(b"\d+", b" ", clean)
    # 6. Remove extra spaces - At the end of previous operations we multiplied space accurences
    clean = re.sub(b'\s+', b' ', clean)
    # Remove ending space if any
    clean = re.sub(b'\s+$', b'', clean)
    # 7. Now replace words by words surrounded by # signs
    # e.g. my name is bond would become #my# #name# #is# #bond#
    # clean = re.sub(b"([a-z]+)", b"#\g<1>#", clean)
    clean = re.sub(b" ", b"# #", clean)  # Replace space
    clean = b"#" + clean + b"#"  # add leading and trailing #

    return str(clean, 'utf-8')

def count_regexp_occ(regexp="", text=None):
    """ Simple way to get the number of occurence of a regex"""
    return len(re.findall(regexp, text))

def get_indicators_and_clean_comments(df, text_var):
    """
    Check all sorts of content as it may help find toxic comment
    Though I'm not sure all of them improve scores
    """
    # Count number of \n
    df["ant_slash_n"] = df[text_var].apply(lambda x: count_regexp_occ(r"\n", x))
    # Get length in words and characters
    df["raw_word_len"] = df[text_var].apply(lambda x: len(x.split()))
    df["raw_char_len"] = df[text_var].apply(lambda x: len(x))
    # Check number of upper case, if you're angry you may write in upper case
    df["nb_upper"] = df[text_var].apply(lambda x: count_regexp_occ(r"[A-Z]", x))
    # Number of F words - f..k contains folk, fork,
    df["nb_fk"] = df[text_var].apply(lambda x: count_regexp_occ(r"[Ff]\S{2}[Kk]", x))
    # Number of S word
    df["nb_sk"] = df[text_var].apply(lambda x: count_regexp_occ(r"[Ss]\S{2}[Kk]", x))
    # Number of D words
    df["nb_dk"] = df[text_var].apply(lambda x: count_regexp_occ(r"[dD]ick", x))
    # Number of occurence of You, insulting someone usually needs someone called : you
    df["nb_you"] = df[text_var].apply(lambda x: count_regexp_occ(r"\W[Yy]ou\W", x))
    # Just to check you really refered to my mother ;-)
    df["nb_mother"] = df[text_var].apply(lambda x: count_regexp_occ(r"\Wmother\W", x))
    # Just checking for toxic 19th century vocabulary
    df["nb_ng"] = df[text_var].apply(lambda x: count_regexp_occ(r"\Wnigger\W", x))
    # Some Sentences start with a <:> so it may help
    df["start_with_columns"] = df[text_var].apply(lambda x: count_regexp_occ(r"^\:+", x))
    # Check for time stamp
    df["has_timestamp"] = df[text_var].apply(lambda x: count_regexp_occ(r"\d{2}|:\d{2}", x))
    # Check for dates 18:44, 8 December 2010
    df["has_date_long"] = df[text_var].apply(lambda x: count_regexp_occ(r"\D\d{2}:\d{2}, \d{1,2} \w+ \d{4}", x))
    # Check for date short 8 December 2010
    df["has_date_short"] = df[text_var].apply(lambda x: count_regexp_occ(r"\D\d{1,2} \w+ \d{4}", x))
    # Check for http links
    df["has_http"] = df[text_var].apply(lambda x: count_regexp_occ(r"http[s]{0,1}://\S+", x))
    # check for mail
    df["has_mail"] = df[text_var].apply(
        lambda x: count_regexp_occ(r'[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+', x)
    )
    # Looking for words surrounded by == word == or """" word """"
    df["has_emphasize_equal"] = df[text_var].apply(lambda x: count_regexp_occ(r"\={2}.+\={2}", x))
    df["has_emphasize_quotes"] = df[text_var].apply(lambda x: count_regexp_occ(r"\"{4}\S+\"{4}", x))

    # Now clean comments
    df["clean_comment"] = df[text_var].apply(lambda x: prepare_for_char_n_gram(x))

    # Get the new length in words and characters
    df["clean_word_len"] = df["clean_comment"].apply(lambda x: len(x.split()))
    df["clean_char_len"] = df["clean_comment"].apply(lambda x: len(x))
    # Number of different characters used in a comment
    # Using the f word only will reduce the number of letters required in the comment
    df["clean_chars"] = df["clean_comment"].apply(lambda x: len(set(x)))
    df["clean_chars_ratio"] = df["clean_comment"].apply(lambda x: len(set(x))) / df["clean_comment"].apply(
        lambda x: 1 + min(99, len(x)))
    
    return df
    
with timer("Performing basic NLP"):
    train = get_indicators_and_clean_comments(train, 'question_text')
    test = get_indicators_and_clean_comments(test,  'question_text')
    
    num_features = [f_ for f_ in train.columns
                if f_ not in ["question_text", "clean_comment", "remaining_chars",
                              'has_ip_address', 'target']]
    

### Cleansing Start

In [7]:
# Cleansing
with timer("Cleansing Dataset"):
    # 並列処理でクレンジング
    train_text_list = Parallel(n_jobs=-1)( [delayed(cleansing_text)(args) for args in train_text_list] )
    train_text_list = Parallel(n_jobs=-1)( [delayed(prepare_for_char_n_gram)(args) for args in train_text_list] )
    train_text_list = Parallel(n_jobs=-1)( [delayed(clean_map_text)(args) for args in train_text_list] )
    train_text_list = Parallel(n_jobs=-1)( [delayed(replace_typical_misspell)(args) for args in train_text_list] )
    
    test_text_list = Parallel(n_jobs=-1)( [delayed(cleansing_text)(args) for args in test_text_list] )
    test_text_list = Parallel(n_jobs=-1)( [delayed(prepare_for_char_n_gram)(args) for args in test_text_list] )
    test_text_list = Parallel(n_jobs=-1)( [delayed(clean_map_text)(args) for args in test_text_list] )
    test_text_list = Parallel(n_jobs=-1)( [delayed(replace_typical_misspell)(args) for args in test_text_list] )
    
    text_list = train_text_list + test_text_list

[Cleansing Dataset] done in 64 s


### TFIDF

In [ ]:
def char_analyzer(text):
    """
    This is used to split strings in small lots
    I saw this in an article (I can't find the link anymore)
    so <talk> and <talking> would have <Tal> <alk> in common
    """
    tokens = text.split()
    return [token[i: i + 3] for token in tokens for i in range(len(token) - 2)]

all_text = pd.concat([train['question_text'],test['question_text']], axis =0)

word_vect = TfidfVectorizer(
            sublinear_tf=True,
            strip_accents='unicode',
            analyzer='word',
            token_pattern=r'\w{1,}',
            stop_words='english',
            ngram_range=(1, 2),
            min_df=10,
            max_df=0.5,
            max_features=20000)

char_vectorizer = TfidfVectorizer(
            sublinear_tf=True,
            strip_accents='unicode',
            tokenizer=char_analyzer,
            analyzer='word',
            ngram_range=(1, 1),
            min_df=10,
            max_df=0.5,
            max_features=50000)

with timer("Word Grams TFIDF"):
    word_vect.fit(all_text)
    train_word_features  = word_vect.transform(train['question_text'])
    test_word_features  = word_vect.transform(test['question_text'])

with timer("Character Grams TFIDF"):
    char_vectorizer.fit(all_text)
    train_char_features = char_vectorizer.transform(train['question_text'])
    test_char_features = char_vectorizer.transform(test['question_text'])

In [ ]:
# Get Sparse Matrix Feature Names..
feature_names = word_vect.get_feature_names() + char_vectorizer.get_feature_names() + num_features
del all_text; gc.collect()

with timer("Sparse Combine"):
    X = hstack(
        [
            train_char_features,
            train_word_features,
            train[num_features]
        ]
    ).tocsr()

    del train_char_features
    gc.collect()

    testing = hstack(
        [
            test_char_features,
            test_word_features,
            test[num_features]
        ]
    ).tocsr()
    del test_char_features; gc.collect()

### LGBM

In [ ]:
train_st_time = time.time()
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import log_loss, confusion_matrix, f1_score, accuracy_score
import eli5
from eli5.sklearn import PermutationImportance
from eli5.permutation_importance import get_score_importances

start_time = "{0:%Y%m%d_%H%M%S}".format(datetime.datetime.now())


def f1_calculation(y_val, y_pred):
    for thresh in np.arange(0.1, 0.401, 0.01):
        thresh = np.round(thresh, 2)
        f1 = f1_score(y_val, (y_pred>thresh).astype(int))
        logger.info(f"F1 score at threshold {thresh} is {f1}")
        
        
# Memory Error
def start_svd(df_tfidf, dim=1000):
    from sklearn.decomposition import TruncatedSVD
    svd = TruncatedSVD(n_components=dim, random_state=seed)
    svd_tfidf = svd.fit_transform(df_tfidf)
#     col_names = [f"svd{dim}_tfidf_{i}" for i in range(dim)]
#     df_svd = pd.DataFrame(svd_tfidf, columns=col_names)
    return svd_tfidf
    

def lgbm_train(train_test, feature_list, fold):

    # LGBM Args
    model_type = 'lgb'
    fold_type = 'stratified'
    metric = 'accuracy'
#     fold = 5
    learning_rate = 0.1
    early_stopping_rounds = 100
    num_boost_round = 10000
    seed = 1208
    params = {
        'num_threads': -1,
        'metric': 'binary_logloss',
        'objective': 'binary',
        'boosting_type':'gbdt',
        'bagging_freq': 1,
        'sigmoid': 1.1,
        'subsample': 0.9,
        'colsample_bytree': 0.2,
        'lambda_l1': 1,
        'lambda_l2': 5,
        'learning_rate': 0.1,
        'num_leaves': 100,
        'max_depth': 9,
        'bagging_seed': 1208,
        'data_random_seed': 1208,
        'feature_fraction_seed': 1208,
        'random_seed': 1208,
        'verbose': 1
    }

    # Result
    select_features = [] # Feature Importanceによって選択したfeature群を入れるリスト
    prediction = np.zeros(len(raw_trn_idx))

    with timer("LGBM Setting"):

        # testも結合されてるので、trainのみにする
#         tmp_train = csr_tfidf[raw_trn_idx]
        tmp_train = train_test[raw_trn_idx]

        ' KFold '
        if fold_type == 'stratified':
            folds = StratifiedKFold(n_splits=fold, shuffle=True, random_state=seed)
            kfold = folds.split(tmp_train, y)

    cv_feim = pd.DataFrame() # Feature Importanceの結果ファイルを入れるDF
    for n_fold, (trn_idx, val_idx) in enumerate(kfold):

        with timer(f"Validation: {n_fold} | LGBM Train"):
            x_train, y_train = tmp_train[trn_idx], y[trn_idx]
            x_val, y_val = tmp_train[val_idx], y[val_idx]

            # Dataset
            lgb_train = lgb.Dataset(data=x_train, label=y_train)
            lgb_eval = lgb.Dataset(data=x_val, label=y_val)

            estimator = lgb.train(
                train_set=lgb_train,
                valid_sets=lgb_eval,
                params=params,
                verbose_eval=200,
                early_stopping_rounds=early_stopping_rounds,
                num_boost_round=num_boost_round
            )
            
#         start_time = time.time()
        
#         def score(X, y):
#             y_pred = estimator.predict(X)
#             return accuracy_score(y, y_pred)
#         base_score, score_decreases = get_score_importances(score, x_train, y_train)
#         feature_importances = np.mean(score_decreases, axis=0)
        
#         from matplotlib import pyplot as plt
#         %matplotlib inline
#         import seaborn as sns
        
#         plt.figure(figsize=(8, 12))
#         sns.barplot(data=[feature_importances[:50], feature_list[:50]], x='avg_importance', y='feature')
#         plt.show()
        
#         total_time = (time.time() - start_time)/60.0
#         print(f"Permutation Importance Calculate: {total_time}")
#         sys.exit()

        with timer(f"Validation: {n_fold} | Prediction & Get F1 score"):
            y_pred = estimator.predict(x_val)
            score = log_loss(y_val, y_pred)
            logger.info(f'Fold No: {n_fold} | {metric}: {score}')
            logger.info(f"Train Shape: {x_train.shape}")
            
            prediction[val_idx] = y_pred
            
            f1_calculation(y_val, y_pred)

            ' Feature Importance '
            if len(cv_feim):
                cv_feim[f'{n_fold}_importance'] = estimator.feature_importance(importance_type='gain')
            else:
                feim_name = f'{n_fold}_importance'
                feim = pd.Series(estimator.feature_importance(importance_type='gain'), name=feim_name, index=feature_list).to_frame().reset_index().rename(columns={'index':'feature'})
                cv_feim = feim.copy()
                
    f1_calculation(y, prediction)

    with timer("Save Feature Importance"):
        col_feim = [col for col in cv_feim.columns if col.count('importance')]
        cv_feim['avg_importance'] = cv_feim[col_feim].mean(axis=1)
        cv_feim.sort_values(by='avg_importance', ascending=False, inplace=True)

        from matplotlib import pyplot as plt
        %matplotlib inline
        import seaborn as sns
        plt.figure(figsize=(8, 12))
        display(cv_feim.head())
        sns.barplot(data=cv_feim.iloc[:50, ], x='avg_importance', y='feature')
        plt.show()
        if is_local:
            cv_feim.to_csv(f'../valid/{start_time[4:12]}_{model_type}_TFIDF_f1{f1}_logloss{score}_lr{learning_rate}.csv', index=False)
        tmp_features = list(cv_feim[cv_feim['avg_importance']>30]['feature'].values)
        print(f'Selece Feature: {len(tmp_features)}')
        select_features += tmp_features

    select_features = list(set(select_features))
    print(len(select_features))
    print(sorted(select_features)[:20])
    print(f'All done in {time.time() - train_st_time:.0f} s')
    
    return select_features, prediction

In [ ]:
tmp_train = csr_tfidf
select_features, _ = lgbm_train(tmp_train, feature_list, fold=2)

### Feature Importance 上位のFeature Set

In [29]:
id_list = []
[id_list.append(id)  for word, id in tfidf_vocablary if word in select_features]
print(len(id_list))
print(len(select_features))
# sparse matrixのcolumn No=id No
select_mx = csr_tfidf.T[id_list].T
print(select_mx.shape)

2509
2509
(1362492, 2509)


In [ ]:
with timer("Selected Train"):
    select_features, prediction = lgbm_train(select_mx, select_features, fold=5)
    print(len(select_features))
    print(select_features)

In [14]:
print(f'All Done!! {time.time() - st_time:.0f} s')

NameError: name 'train_st_time' is not defined